In [16]:
import pandas as pd
from smart_open import open
import boto3
from tqdm import tqdm
import numpy as np

# Import credentials
import credentials

In [17]:
# Create boto3 session
session = boto3.session.Session()

# Transport params for smart_open
transport_params = {
    "client": session.client(
        "s3",
        endpoint_url=credentials.MINIO_URL,
        aws_access_key_id=credentials.ACCESS_KEY,
        aws_secret_access_key=credentials.SECRET_KEY
    )
}

In [18]:
# Load your opioid-related PcrKeys
pcr_keys_df = pd.read_csv("../data/interim/opioid_pcr_keys.csv")

# Convert to set for fast lookup
pcr_key_set = set(pcr_keys_df["PcrKey"].astype(str))

print("Loaded PcrKeys:", len(pcr_key_set))

Loaded PcrKeys: 271206


In [19]:
# Path to the event data file
event_uri = "s3://{}/raw-data/pub_pcrevents_cp25.txt".format(credentials.BUCKET_NAME)

# Prepare chunks
event_chunks = []

with open(event_uri, transport_params=transport_params) as f:
    for chunk in tqdm(
        pd.read_csv(f, delimiter="|", chunksize=100_000),
        desc="Loading Event Records"
    ):
        chunk.columns = chunk.columns.str.strip(" ~'")
        chunk["PcrKey"] = chunk["PcrKey"].astype(str).str.strip(" ~")

        # Filter to only your subset of PcrKeys
        filtered = chunk[chunk["PcrKey"].isin(pcr_key_set)]
        if not filtered.empty:
            event_chunks.append(filtered)

# Combine all filtered chunks
event_df = pd.concat(event_chunks, ignore_index=True)

print("Loaded event records:", len(event_df))

Loading Event Records: 5it [36:53, 442.62s/it]


EndpointConnectionError: Could not connect to the endpoint URL: "http://server.jakemccaig.com:9768/nemsis-data/raw-data/pub_pcrevents_cp25.txt"